In [1]:
import pandas as pd
import numpy as np
import scipy as sci
import math
import time
from tools.auxiliary import*
from numpy.linalg import inv

#below will be merged later to auxilary
from tools.GH_Quadrature import*
from tools.Ord_Herm_Pol_1 import*
from tools.Num_Stab_Approx import*

In [2]:
df =reading("epsi10000.csv")
T  = 10000           #Choose the simulation length for the solution procedure, T<=10,000                    
gam     = 1        # Utility-function parameter
alpha   = 0.36     # Capital share in output
beta    = 0.99     # Discount factor
delta   = 0.02     # Depreciation rate 
rho     = 0.95     # Persistence of the log of the productivity level
sigma   = 0.01    # Standard deviation of shocks to the log of the productivity level
ks = ( (1-beta+beta*delta) / (alpha*beta) )**(1/(alpha-1) )
k = np.array([ks]*(T+1))
a= [1]*(T)
epsi = df.iloc[:,0].astype(float)*sigma
epsi=epsi.tolist()
for i in range(1, T):
    a[i]=a[i-1]**(rho)*math.exp(epsi[i])
a=np.asarray(a)
kdamp = 0.01    
dif_GSSA_1d = 1e+10  
bk_1d  = np.array([0., 0.95, ks*0.05])
bk_1d= np.reshape(bk_1d, (3,1))
k_old = [ks+1]*(T+1)

Target:

In [3]:
def GSSA_main_cycle(T, gam, alpha, beta, delta, kdamp, dif_GSSA_1d, a, bk_1d, k_old, checker = 0):
    '''
    Compute a first-degree polynomial solution using the one-node Monte Carlo integration method.

            Parameters:
                    T (int): A decimal integer
                    gam (int): Another decimal integer
                    alpha:
                    beta:
                    delta: 
                    kdamp: 
                    dif_GSSA_1d: 
                    a: 
                    bk_1d:
                    k_old: 
                    checker (int): A checker for the while loop, used to validate if the loop is running correctly or not.

            Returns:
                    numpy array: A numpy array of floats with the shape of (9999,)
    '''
    while dif_GSSA_1d > 1e-4*kdamp:
        x=[]
        for i in range(0,T):
            x.append([1,k[i],a[i]])
            k[i+1]=np.matmul(x[i], bk_1d)
        x=np.asarray(x)
        c= np.multiply(np.power(k[0:T], alpha),a)+ (1-delta)*k[0:T]-k[1:T+1]
        y= np.multiply(np.divide(beta*np.power(c[1:T],-gam),np.power(c[0:T-1], -gam))*(1-delta+np.multiply(alpha*np.power(k[1:T],(alpha-1)),a[1:T])),k[1:T])
        dif_GSSA_1d = np.mean(abs(1-np.divide(k,k_old)))
        if checker == 1:
            print(dif_GSSA_1d) #for values checking
        bk_hat_1d= np.matmul(inv(np.matmul(np.transpose(x[0:T-1,:]),x[0:T-1,:])),np.matmul(np.transpose(x[0:T-1,:]),y[0:T-1]))
        bk_1d = (kdamp*bk_hat_1d+ (1-kdamp)*bk_1d.reshape(1,3)).reshape(3,1)
        k_old=np.copy(k)
    return y

In [4]:
y = GSSA_main_cycle(T, gam, alpha, beta, delta, kdamp, dif_GSSA_1d, a, bk_1d, k_old)

In [5]:
#The GSSA parameters

kdamp = 0.1
dif_GSSA_D = 1e+10

#The matrices of the polynomial coefficients
D_max  = 5 #because of python
npol = np.array([3, 6, 10, 15, 21])

BK = np.zeros((npol[D_max-1], D_max))

# 13. Choose an integration method for computing solutions  
IM  = 10

n_nodes,epsi_nodes, weight_nodes= GH_Quadrature(Qn=10, N=1, vcv=sigma**2)

#make sure to change a into the right shape
a = np.reshape(a, (T, 1))
a1 = np.matmul(np.power(a,rho), np.exp(epsi_nodes.transpose()))

#14. Choose a regression specification 
RM = 6           # Choose a regression method: 
                 # 1=OLS,          2=LS-SVD,   3=LAD-PP,  4=LAD-DP, 
                 # 5=RLS-Tikhonov, 6=RLS-TSVD, 7=RLAD-PP, 8=RLAD-DP
normalize = 1    # Option of normalizing the data; 0=unnormalized data; 
                 # 1=normalized data                    
penalty = 7      # Degree of regularization for a regularization methods, 
                 # RM=5,6,7,8 (must be negative, e.g., -7 for RM=5,7,8 
                 # and must be positive, e.g., 7, for RM=6)
PF = 0           # Choose a polynomial family; 0=Ordinary (default);  
                 # 1=Hermite
# 15. Initialize the capital series
zb = np.matrix([[np.mean(k[0:T]), np.mean(a[0:T])], [np.std(k[0:T]), np.std(a[0:T])]])
z = np.concatenate((k[0:T].reshape(T,1), a[0:T].reshape(T,1)), axis=1)
k_old = [ks+1]*(T+1)

In [6]:
d= 1

In [7]:
X = Ord_Herm_Pol_1(z=z, D=d, PF=PF, zb=zb)
bk_D = Num_Stab_Approx(X[0:T-1,:],y.reshape((y.shape[0],1)),RM,penalty,normalize)
while dif_GSSA_D > 1e-4/10**(d*kdamp):
    for i in range(T):
        X[i]= Ord_Herm_Pol_1(np.array([k[i],float(a[i])]).reshape((1,2)), D=d, PF=PF, zb=zb)
        k[i+1] = np.matmul(X[i], bk_D)
    c = np.multiply(np.power(k[0:T], alpha).reshape((T,1)), a) + (1-delta)*k[0:T].reshape(T,1)-k[1:T+1].reshape(T,1)
    k1 = k[1:T+1]
    c1 = np.zeros((T,n_nodes))
    if IM == 0:
        y = np.multiply(np.divide(beta*np.power(c[1:T],-gam),np.power(c[0:T-1], -gam))*(1-delta+np.multiply(alpha*np.power(k[1:T],(alpha-1)).reshape(T-1,1),a[1:T])),k[1:T].reshape(T-1, 1))
    else:
        for i in range(n_nodes):
            k2_prep = np.concatenate((k1.reshape(T,1), a1[:,i].reshape(T,1)), 1)  #preparation for k2 @
            k2= np.matmul(Ord_Herm_Pol_1(k2_prep,d,PF,zb), bk_D)  #1 equals D
            c1[:,i] = np.ravel((np.multiply(np.power(k1, alpha),a1[:,i])+ (1-delta)*k1).reshape(T,1) - k2)
        k1_dupl = np.tile(k1, (n_nodes,1)).transpose() 
        c_dupl = np.tile(np.ravel(c), (n_nodes,1)).transpose()
        y = np.matmul(np.multiply(np.divide(beta*np.power(c1,-gam),np.power(c_dupl, -gam))*(1-delta+np.multiply(alpha*np.power(k1_dupl,(alpha-1)),a1)),k1_dupl), weight_nodes)
    
    dif_GSSA_D = np.mean(abs(1-np.divide(k,k_old)))
    print(dif_GSSA_D) #for values checking
    bk_hat_D = Num_Stab_Approx(X[0:T-1,:],y[0:T-1,:],RM,penalty,normalize)
    bk_D = (kdamp*bk_hat_D+ (1-kdamp)*bk_D)
    k_old=np.copy(k)

0.035193270265045595
0.00012850550683394296
0.00011964777526948686
0.00011145127452404137
0.00010385937142510122
9.682124228113684e-05
9.029116532016695e-05
8.422791081096829e-05
7.85942141060351e-05


In [ ]:
def 

In [19]:
c = np.multiply(np.power(k[0:T], alpha).reshape((T,1)), a) + (1-delta)*k[0:T].reshape(T,1)-k[1:T+1].reshape(T,1)
k1 = k[1:T+1]
c1 = np.zeros((T,n_nodes))

Problem:

In [22]:
for i in range(n_nodes):
            k2_prep = np.concatenate((k1.reshape(T,1), a1[:,i].reshape(T,1)), 1)  #preparation for k2 @
            k2= np.matmul(Ord_Herm_Pol_1(k2_prep,d,PF,zb), bk_D)  #1 equals D
            c1[:,i] = np.ravel((np.multiply(np.power(k1, alpha),a1[:,i])+ (1-delta)*k1).reshape(T,1) - k2)

In [13]:
a1[:,9]

array([0.9525672 , 0.93998278, 0.93450804, ..., 0.8995193 , 0.90978163,
       0.90539921])

In [17]:
test = (np.multiply(np.power(k1, alpha),a1[:,9])+ (1-delta)*k1).reshape(T,1) - k2
#9 needs to be replace to i

In [23]:
k1_dupl = np.tile(k1, (n_nodes,1)).transpose() 
c_dupl = np.tile(np.ravel(c), (n_nodes,1)).transpose()
y = np.matmul(np.multiply(np.divide(beta*np.power(c1,-gam),np.power(c_dupl, -gam))*(1-delta+np.multiply(alpha*np.power(k1_dupl,(alpha-1)),a1)),k1_dupl), weight_nodes)